<h1 style="color:purple;font-size:30px;" ><center>iAvenue Farm Layers</center></h1> 

In [1]:
# Authentication of Earth engine
import ee
service_account = 'iavenuelabs@ialabs-earthengineapi.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'ialabs-earthengineapi-df98e792a65f.json')
ee.Initialize(credentials)

In [2]:
# Importing Libraries
import os
import ee
import geemap
import ipywidgets as widgets
import os
import subprocess
import sys
from ipywidgets import FileUpload
from IPython.display import Image
import requests
import os
from ftplib import FTP
import numpy as np
from PIL import Image 
from IPython.display import display, FileLink
from ipywidgets import HTML, Layout,VBox

In [3]:
Map = geemap.Map(center=(20.5937, 78.9629), zoom=14)
Map.add_basemap('HYBRID')
# Map

In [4]:
ndvi_button = widgets.Button(
    description='Calculate NDVI',
    button_style='primary',
    tooltip='Click to Calculate NDVI',
    #layout=Layout(width='50%', height='80px')
    layout=Layout(height='auto', width='auto')
)

output = widgets.Output()

hbox5 = widgets.HBox([ndvi_button])
ndvi_button.style.button_color = 'purple'

In [23]:
def ndvi_clicked(b):

    with output:
        output.clear_output()
        a = Map.user_roi.getInfo()
        aoi1=a['coordinates']
        point = ee.Geometry.Polygon(aoi1)
        #l8 = ee.ImageCollection('MODIS/MCD43A4_006_NDVI')
        l8 = ee.ImageCollection('COPERNICUS/S2_SR')
        #l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
        image = ee.Image(
            l8.filterBounds(point)
            .filterDate('2021-04-05', '2021-04-10')
            .sort('CLOUD_COVER')
            .first()
        )
        # Compute NDVI.
        NIR = image.select('B8')
        Red = image.select('B4')
        ndvi1999 = NIR.subtract(Red).divide(NIR.add(Red)).rename('NDVI')
        ndvi1999=ndvi1999.clip(point)
        #s2NDVI = ndvi1999.select('ndvi');
        Map.addLayer(ndvi1999, {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}, "NDVI 1999")
        Map 
        print("NDVI Calculated")
        area = point.area()
        #print("area",area)
        area_ha=area.getInfo()
        area_ha=area_ha/4047
        #print("ha",area_ha)
        #print("ROI Area = ", round(area.getInfo(),2),"Square meter")
        print("ROI Area = ", round(area_ha,2),"Acre")

        
ndvi_button.on_click(ndvi_clicked)

In [6]:
def on_upload_change(change):
    global file
    if not change.new:
        return
    up = change.owner
    for filename,data in up.value.items():
        with open(filename, 'wb') as f:
            f.write(data['content'])            
    up.value.clear()
    up._counter = 0
    file = filename
upload_btn = FileUpload(
    #layout=widgets.Layout(width='50%', height='80px')
    layout=Layout(height='auto', width='auto')

)
upload_btn.observe(on_upload_change, names='_counter')
upload_btn.style.button_color = 'purple'


In [7]:
snip_button = widgets.Button(
    description='Snip Area of Interest',
    button_style='primary',
    tooltip='Click to Snip Image',
    #layout=Layout(width='50%', height='80px')
    layout=Layout(height='auto', width='auto')
)

output2 = widgets.Output()

hbox5 = widgets.HBox([snip_button])
snip_button.style.button_color = 'purple'

In [8]:
def snip_clicked(a):
    with output2:
        global file
        output2.clear_output()
        subprocess.run("dir",shell=True)
        subprocess.run([sys.executable, 'snipping_tool.py', '--source',])
        file = 'iaLabs123.png'
        print("Area of Interest Snipped")
        

snip_button.on_click(snip_clicked)

In [9]:
send_button = widgets.Button(
    description='Send Image',
    button_style='primary',
    tooltip='Click to Send Image',
    #layout=Layout(width='50%', height='80px')
    layout=Layout(height='auto', width='auto')
)


output1 = widgets.Output()

hbox5 = widgets.HBox([send_button])
send_button.style.button_color = 'purple'

In [10]:
def send_clicked(c):
    with output1:
        output1.clear_output()
        
        print("Calculating")  
        print("File name = ",file)
        subprocess.run("dir",shell=True)
        subprocess.run([sys.executable, 'detect.py', '--source', os.path.join((file))])
        farm_file = open("farm.txt", "r")
        fdata = farm_file.read()
        print("Total No of Farms in the selected ROI are: ",fdata)
        farm_file.close()
        local_file = FileLink("./static/" + file , result_html_prefix = "Click here to download: ")
        display(local_file) 
        output1
        
        
send_button.on_click(send_clicked)

In [11]:
header = widgets.Button(description="Header",
                    layout=widgets.Layout(width="auto", height="auto"), button_style="success")
footer = widgets.Button(description="Footer",
                    layout=widgets.Layout(width="auto", height="auto"), button_style="primary")
left = widgets.Button(description="Left",
                    layout=widgets.Layout(width="auto", height="auto"), button_style="info")
right = widgets.Button(description="Right",
                    layout=widgets.Layout(width="auto", height="auto"), button_style="info")
center = widgets.Button(description="Center",
                    layout=widgets.Layout(width="auto", height="auto"), button_style="danger")

In [12]:
#widgets.AppLayout(
#          left_sidebar=Map,
#          right_sidebar=VBox([ndvi_button,output,upload_btn,snip_button,output2,send_button,output1]),
#            footer = VBox([output]),
#           grid_gap="40px",
#          )

In [13]:
from ipywidgets import AppLayout, Button, Layout

In [14]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

In [15]:
header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

In [16]:
top_left_button = create_expanded_button("Top left", 'info')
top_right_button = create_expanded_button("Top right", 'success')
bottom_left_button = create_expanded_button("Bottom left", 'danger')
bottom_right_button = create_expanded_button("Bottom right", 'warning')

In [17]:
from ipywidgets import TwoByTwoLayout



In [20]:
AppLayout(header=None,
          left_sidebar=None,
          center=Map,
          right_sidebar=VBox([output,output2,output1]),
          footer=TwoByTwoLayout(top_left=ndvi_button,
               top_right=upload_btn,
               bottom_left=snip_button,
               bottom_right=send_button))

AppLayout(children=(TwoByTwoLayout(children=(Button(button_style='primary', description='Calculate NDVI', layo…

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
